In [1]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

c:\program files (x86)\microsoft visual studio\shared\python37_64\lib\site-packages\ipykernel_launcher.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  after removing the cwd from sys.path.
c:\program files (x86)\microsoft visual studio\shared\python37_64\lib\site-packages\ipykernel_launcher.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  """


In [2]:
spacex_df = pd.read_csv("spacex_launch_dash.csv")

In [3]:
print(spacex_df.shape)

(56, 6)


In [4]:
spacex_df.head(20)

,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0
5,7,CCAFS LC-40,0,3170.0,F9 v1.1,v1.1
6,8,CCAFS LC-40,0,3325.0,F9 v1.1,v1.1
7,9,CCAFS LC-40,0,2296.0,F9 v1.1,v1.1
8,10,CCAFS LC-40,0,1316.0,F9 v1.1,v1.1
9,11,CCAFS LC-40,0,4535.0,F9 v1.1,v1.1


In [ ]:


# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                 dcc.Dropdown(id='site-dropdown',
                                    options=[
                                        {'label': 'All Site', 'value': 'ALL'},
                                        {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                        {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                        {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                        {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                    ],
                                    value='ALL',
                                    placeholder="Select a Launch Site here",
                                    searchable=True
                                    ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                    min=0, max=10000, step=1000,
                                    marks={0: '0',
                                        100: '100'},
                                    value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart'))
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):
    #This function adjust the pie chart for the selecting in the Dropdown object. The entire value is the site name or la option "ALL"
    if entered_site == 'ALL': #in this case we not need put any filter and our pie chart has class like values and Launch Site like names
        fig = px.pie(spacex_df, values='class', 
        names='Launch Site', 
        title='title ALL Site')
        return fig
    else: 
        #we selecting any site (not ALL)
        site_fil=pd.DataFrame() #we will use a dataframe auxiliary
        the_fil = spacex_df['Launch Site'] == entered_site #we create a filter with the site selected and put it in the site_fil
        site_fil =spacex_df[the_fil]
        # in this case we omit the label "values" so pie chart express the successful and unsuccessfully launch
        fig = px.pie(site_fil, 
        names='class', 
        title= 'title '+ str(entered_site))
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'), Input(component_id="payload-slider", component_property="value")])
  
def get_scatter_plot(entered_site, entered_payload):
    low, high = entered_payload # the SlideRange gives two values: the low value and high value, here put them in this variables
    #we create the first filter "mask" what delimiting between both values
    mask = (spacex_df['Payload Mass (kg)'] > low) & (spacex_df['Payload Mass (kg)'] < high) 
    if entered_site == 'ALL':
        # for all sites we apply the filter mask 
        fig = px.scatter(spacex_df[mask], x='Payload Mass (kg)', y='class', color='Booster Version Category')
        return fig
    else:
        the_fil = spacex_df['Launch Site'] == entered_site #we create de second filter for the site selected
        site_fil=pd.DataFrame() #again we will use a dataframe auxiliary
        site_fil =spacex_df[mask] #we apply the first filter
        site_fil =site_fil[the_fil] #and the second filter
        fig = px.scatter(site_fil, x='Payload Mass (kg)', y='class', color='Booster Version Category')
        return fig



# Run the app
if __name__ == '__main__':
    app.run_server()


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [22/May/2023 16:51:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/May/2023 16:51:01] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [22/May/2023 16:51:01] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [22/May/2023 16:51:01] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [22/May/2023 16:51:01] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [22/May/2023 16:51:01] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [22/May/2023 16:51:01] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 304 -
127.0.0.1 - - [22/May/2023 16:51:03] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/May/2023 16:51:03] "POST /_dash-update-component HTTP/1.1" 200 -
c:\program files (x86)\microsoft visual studio\shared\python37_64\lib\site-packages\ipykernel_launcher.py:90: UserWarning:

Boolean Seri

In [ ]:
spacex_df['class']

In [ ]:
max_payload

In [ ]:
min_payload